In [ ]:
import os

def get_aws_credentials():
    access_key = os.environ.get("MY_ACCESS_KEY")
    secret_access_key = os.environ.get("MY_SECRET_ACCESS_KEY")
    region = os.environ.get("AWS_REGION")
    bucket_name = os.environ.get("BUCKET_NAME")
    return access_key, secret_access_key, region, bucket_name

In [ ]:
import json
import boto3
import csv
from credentials import get_aws_credentials

def lambda_handler(event, context):
    # Get AWS credentials and bucket name
    access_key, secret_access_key,region, bucket_name = get_aws_credentials()

    # Create an S3 client
    s3 = boto3.client('s3',aws_access_key_id=access_key, aws_secret_access_key=secret_access_key, region_name=region)

    try:
        response = s3.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response:
            files_content = {}
            for obj in response['Contents']:
                file_key = obj['Key']
                print("File:", file_key)
                # Get the contents of the file
                try:
                    file_obj = s3.get_object(Bucket=bucket_name, Key=file_key)
                    file_content = file_obj['Body'].read().decode('utf-8')
                    # Parse CSV content
                    csv_reader = csv.reader(file_content.splitlines())
                    file_data = []
                    for row in csv_reader:
                        file_data.append(row)
                    files_content[file_key] = file_data
                except Exception as e:
                    print("Error:", e)
            return {
                'statusCode': 200,
                'headers': {
                    'Content-Type': 'application/json',
                    'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
                    'Access-Control-Allow-Methods': 'POST',
                    'Access-Control-Allow-Origin': '*'
                },
                'body': json.dumps(files_content)
            }
        else:
            print("No files found in the bucket.")
            return {
                'statusCode': 404,
                'headers': {
                    'Content-Type': 'application/json',
                    'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
                    'Access-Control-Allow-Methods': 'POST',
                    'Access-Control-Allow-Origin': '*'
                },
                'body': json.dumps({"message": "No files found in the bucket."})
            }
    except Exception as e:
        print("Error:", e)
        return {
            'statusCode': 500,
            'headers': {
                'Content-Type': 'application/json',
                'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
                'Access-Control-Allow-Methods': 'POST',
                'Access-Control-Allow-Origin': '*'
            },
            'body': json.dumps({"message": "Internal server error."})
        }

